In [58]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import os
import numpy as np

In [59]:
iceCount = list()
averageTemp = list()
particleID = list()

In [60]:
def createIceParticleID(data):
#     print(data)
    dataSortedByIce = data[data.iceLabel == 1]
    particleID.append(list(dataSortedByIce.origId))

In [61]:
path = 'contrails raw data/contrails 3/'
for filename in glob.glob(os.path.join(path, '*.csv')):
   with open(os.path.join(os.getcwd(), filename), 'r') as f: # open in readonly mode
      # do your stuff
        contrails = pd.read_csv(f)
        createIceParticleID(contrails)
        valueCountIce = list(contrails.iceLabel.value_counts())
        averageTemp.append(round(contrails["T"].mean(),2))
        iceCount.append(valueCountIce)
        

In [62]:
newIceParticles = list()
# adding the first time steps
newIceParticles.append(len(particleID[0]))
print(newIceParticles)
for i in range(0, len(particleID)-1):
    count = 0
    for j in range(0, len(particleID[i])):
#         print(count)
        if(particleID[i][j] in particleID[i+1]):
            count = count + 1
    newIceParticles.append(len(particleID[i+1]) - count)
        

[5]


In [63]:
newIceParticles

[5, 149, 405, 403, 424, 413, 442, 427, 391, 436, 444, 427, 424, 446, 390]

In [64]:
iceCount

[[419, 5],
 [695, 154],
 [715, 559],
 [961, 740],
 [1384, 741],
 [1465, 752],
 [1481, 730],
 [1483, 727],
 [1441, 761],
 [1452, 755],
 [1463, 738],
 [1460, 746],
 [1482, 749],
 [1499, 730],
 [1456, 762]]

In [65]:
averageTemp

[601.78,
 456.71,
 385.37,
 348.25,
 323.24,
 318.09,
 317.22,
 318.84,
 319.95,
 320.23,
 318.31,
 319.69,
 318.65,
 318.17,
 318.81]

In [66]:
decreaseInTemp = [0]
for i in range(1, len(averageTemp)):
    decreaseInTemp.append(round(averageTemp[i-1]) - round(averageTemp[i]))
    
decreaseInTemp

[0, 145, 72, 37, 25, 5, 1, -2, -1, 0, 2, -2, 1, 1, -1]

In [67]:
extractedData = pd.DataFrame(iceCount, columns=["noIce","Ice"])
total = list()
percentage = list()
timesteps = np.round(np.arange(0.06,0.2,0.01),2).tolist()
# timesteps
for index, row in extractedData.iterrows():
    sumParticle = row["noIce"] + row["Ice"]
    total.append(round(sumParticle))
    percentage.append(round((row["Ice"]/sumParticle)*100,2))
    
extractedData.insert(0, "Timesteps", timesteps, True)
extractedData.insert(1, "TotalParticles", total, True)
extractedData["NewIce"] = newIceParticles
extractedData["TotalIcePercentage"] = percentage
extractedData["Temp"] = averageTemp


In [68]:
remainingIceParticles = list()
newIcePercentage = list()
remainingIceParticles.append(total[0])
newIcePercentage.append(round((iceCount[0][1]/total[0])*100,2))
for t in range(1, len(total)):
    temp = total[t] - iceCount[t-1][1]
    remainingIceParticles.append(temp)
    newIcePercentage.append(round((newIceParticles[t]/temp)*100,2))
    

extractedData["TotalNonIce"] = remainingIceParticles
extractedData["newIcePercentage"] = newIcePercentage


In [69]:
 
extractedData

,Timesteps,TotalParticles,noIce,Ice,NewIce,TotalIcePercentage,Temp,TotalNonIce,newIcePercentage
0,0.06,424,419,5,5,1.18,601.78,424,1.18
1,0.07,849,695,154,149,18.14,456.71,844,17.65
2,0.08,1274,715,559,405,43.88,385.37,1120,36.16
3,0.09,1701,961,740,403,43.50,348.25,1142,35.29
4,0.10,2125,1384,741,424,34.87,323.24,1385,30.61
5,0.11,2217,1465,752,413,33.92,318.09,1476,27.98
6,0.12,2211,1481,730,442,33.02,317.22,1459,30.29
7,0.13,2210,1483,727,427,32.90,318.84,1480,28.85
8,0.14,2202,1441,761,391,34.56,319.95,1475,26.51
9,0.15,2207,1452,755,436,34.21,320.23,1446,30.15


In [70]:
extractedData.to_csv ('Extracted Data/contrails 3.csv', index = False, header=True)